In [37]:
#nltk.download()
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [28]:
df = pd.read_csv("Reviews.csv")
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [29]:
df['Text'] = df['Text'].str.lower()
df['Text']

0         i have bought several of the vitality canned d...
1         product arrived labeled as jumbo salted peanut...
2         this is a confection that has been around a fe...
3         if you are looking for the secret ingredient i...
4         great taffy at a great price.  there was a wid...
                                ...                        
568449    great for sesame chicken..this is a good if no...
568450    i'm disappointed with the flavor. the chocolat...
568451    these stars are small, so you can give 10-15 o...
568452    these are the best treats for training and rew...
568453    i am very satisfied ,product is as advertised,...
Name: Text, Length: 568454, dtype: object

In [30]:
df['Text'] = df['Text'].apply(word_tokenize)
df['Text']

0         [i, have, bought, several, of, the, vitality, ...
1         [product, arrived, labeled, as, jumbo, salted,...
2         [this, is, a, confection, that, has, been, aro...
3         [if, you, are, looking, for, the, secret, ingr...
4         [great, taffy, at, a, great, price, ., there, ...
                                ...                        
568449    [great, for, sesame, chicken, .., this, is, a,...
568450    [i, 'm, disappointed, with, the, flavor, ., th...
568451    [these, stars, are, small, ,, so, you, can, gi...
568452    [these, are, the, best, treats, for, training,...
568453    [i, am, very, satisfied, ,, product, is, as, a...
Name: Text, Length: 568454, dtype: object

In [31]:
stop_words = set(stopwords.words("english"))
print(stop_words)

{"you'll", 'between', 'most', 'from', 'no', 'these', 't', 'after', 'will', 'now', "it's", 'ourselves', 'aren', 'himself', 'in', 'he', 'more', 'nor', 'o', 'off', 'doesn', 'below', 'there', 'wasn', 'itself', 'mustn', 'again', 'can', 'theirs', 'of', 'didn', 'hasn', 'until', "haven't", 'shan', 'any', "you'd", 'having', 'only', "didn't", 'hers', 'while', 'too', 'with', 'my', 'doing', 'they', 'won', 'weren', 's', 'so', 'under', 'and', 'd', 'it', 'that', 'which', 'haven', 'yourselves', "mustn't", 'had', 'does', "hasn't", 'should', 'into', 'where', 'very', 'not', 'hadn', "that'll", 'am', 'few', 'is', 'myself', 'his', 'been', 'don', 'do', 'themselves', 'how', 'then', 'him', 'what', 'other', 'same', "mightn't", 'all', 're', 'some', 'by', 'own', 'your', 'her', 'above', 'ma', 'if', "shouldn't", 'further', 'on', "weren't", 'who', 'them', "shan't", 'during', "won't", 'this', 'against', 'shouldn', 'm', 'you', 'each', 'me', 'are', 'being', 'their', 'at', 'both', 'a', 'about', 'just', 'wouldn', "she's"

In [32]:
for i in range(len(df)):
    words = df.loc[i, 'Text']
    filtered = []
    for w in words:
        if w.isalpha() and w not in stop_words:
            filtered.append(w)
        
    df.at[i, 'Text'] = filtered

df['Text']

0         [bought, several, vitality, canned, dog, food,...
1         [product, arrived, labeled, jumbo, salted, pea...
2         [confection, around, centuries, light, pillowy...
3         [looking, secret, ingredient, robitussin, beli...
4         [great, taffy, great, price, wide, assortment,...
                                ...                        
568449    [great, sesame, chicken, good, better, restura...
568450    [disappointed, flavor, chocolate, notes, espec...
568451    [stars, small, give, one, training, session, t...
568452    [best, treats, training, rewarding, dog, good,...
568453    [satisfied, product, advertised, use, cereal, ...
Name: Text, Length: 568454, dtype: object

In [33]:
for i in range(len(df)):
    words = df.at[i, 'Text']
    joined_words = " ".join(words)
    df.at[i, 'Text'] = joined_words

df['Text']

0         bought several vitality canned dog food produc...
1         product arrived labeled jumbo salted peanuts p...
2         confection around centuries light pillowy citr...
3         looking secret ingredient robitussin believe f...
4         great taffy great price wide assortment yummy ...
                                ...                        
568449    great sesame chicken good better resturants ea...
568450    disappointed flavor chocolate notes especially...
568451    stars small give one training session tried tr...
568452    best treats training rewarding dog good groomi...
568453    satisfied product advertised use cereal raw vi...
Name: Text, Length: 568454, dtype: object

In [34]:
df = df[df['Score']!=3].reset_index(drop=True)

df.loc[:, 'Target'] = 0

for i in range(len(df)):
    if df.loc[i, 'Score'] > 3:
        df.loc[i, 'Target'] = 1

In [35]:
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(df['Text'])

print(X.shape)

(525814, 99305)


In [36]:
y = df['Target']

In [38]:
X_train , X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [39]:
model = LogisticRegression()

In [40]:
model.fit(X_train, y_train)

LogisticRegression()

In [41]:
y_pred = model.predict(X_test)

In [42]:
print(f"Accuracy: {accuracy_score(y_test, y_pred)*100}%")
print(f"Precision: {precision_score(y_test, y_pred)*100}%")
print(f"Recall: {recall_score(y_test, y_pred)*100}%")
print(f"F1 score: {f1_score(y_test, y_pred)*100}%")

Accuracy: 93.53955288456966%
Precision: 94.59080229725026%
Recall: 97.94895476662462%
F1 score: 96.24059318282427%
